In [2]:
import file
import scoring

Here's how the original and reshaped data look:

In [7]:
data = file.WideData('Measurement study/Participant level data/AlternateUses.xls')
data._original.sample(1)

,participant,book,rope,fork,table,pants,bottle,brick,tire,shovel,shoe
70,71,Doorstop\nRead\nPress flowers\nPaperweight\nPr...,Tie up dog\nHold a robe closed\nHold pants up\...,Eat\nPry a jar lid open\nDig in the dirt\nScra...,Eat dinner\nHold important papers\nDo homework...,Wear them\nTie around your head for sunshade. ...,Hold liquid\nHold fireflies\nEmergency pee bot...,Build something\nThrow at something scary\nSta...,Drive on it\nRoll it down a hill for fun\nPlan...,Dig flowers\nChop the head off a snake\nDig a ...,Squash a bug\nWear on your feet\nDance\nHold a...


In [8]:
data.df.sample(2)

,participant,prompt,response_num,response
11,12,book,0,as a weapon
8598,43,table,9,to sell for money


## Scoring

Scoring is done with the AUT_Scorer class.

In [ ]:
scorer = scoring.AUT_Scorer()
scorer.load_model('EN_100_lsa', '/data/tasa/EN_100k.word2vec.bin')
scorer.load_model('TASA_lsa', '/data/tasa/TASA.word2vec.bin')

In [11]:
scorer.originality('shoe', 'wear', 'EN_100_lsa')

0.31790173053741455

In [13]:
scorer.originality('shoe', 'planter', 'EN_100_lsa')

0.6076052188873291

## Scoring at the Dataset Level

The WideData class takes a scorer and will add an internal column:

In [14]:
data.score(scorer, 'EN_100_lsa', stop=True, idf=True)
data.df.sample(3)

,participant,prompt,response_num,response,EN_100_lsa_stop_idf
1629,66,tire,1,as a shield,0.514952
627,76,brick,0,a weapon,0.772552
2802,43,book,3,doorstop,NaN


You can also have it automatically score all the models that the scorer has loaded.

In [16]:
data.score_all(scorer)

Scoring EN_100_lsa
Column EN_100_lsa_stop_idf already exists. Re-crunching and re-writing.
Scoring TASA_lsa


In [28]:
data.df.sample(2)

,participant,prompt,response_num,response,EN_100_lsa_stop_idf,TASA_lsa_stop_idf
3836,65,rope,4,make a pencil holder,0.540625,0.963029
84,85,book,0,read,0.246462,0.727653
